# ULKB Initialization

In [12]:
from ulkb import *                   # import ULKB Logic namespace
settings.serializer.ulkb.ensure_ascii = False

# FOL reasoning

In [13]:
a             = TypeVariable('a') # a : *
ab            = FunctionType(a, BoolType()) # a -> bool : *
aab           = FunctionType(a, a, BoolType()) # a -> a -> bool : *

lives         = new_constant("lives", ab)
killed        = new_constant("killed", aab)
hates         = new_constant("hates", aab)
richer        = new_constant("richer", aab)

agatha        = new_constant('agatha', a)
butler        = new_constant('butler', a)
charles       = new_constant('charles', a)

x, y = Variables('x', 'y', a)

ax1  = Exists(x, And(lives(x), killed(x,agatha)))
ax3  = Forall(x, Implies(lives(x),Or(equal(x,agatha),equal(x,butler),equal(x,charles))))
ax4  = Forall(x, Forall(y, Implies(killed(x,y),hates(x,y))))
ax5  = Forall(x, y, Implies(killed(x,y), Not(richer(x,y))))
ax6  = Forall(x, Implies(hates(agatha,x), Not(hates(charles,x))))
ax7  = Forall(x, Implies(Not(equal(x,butler)), hates(agatha,x)))
ax8  = Forall(x, Implies(Not(richer(x, agatha)), hates(butler,x)))
ax9  = Forall(x, Implies(hates(agatha,x), hates(butler,x)))
ax10 = Not(Exists(x, Forall(y, Implies(lives(y),hates(x,y)))))
ax11 = Not(equal(agatha,butler))

for ax in [ax1,ax3,ax4,ax5,ax6,ax7,ax8,ax9,ax10,ax11]: 
    new_axiom(ax)

show_axioms()

26	axiom ∃ x, lives x ∧ killed x agatha
27	axiom ∀ x, lives x → x = agatha ∨ x = butler ∨ x = charles
28	axiom ∀ x y, killed x y → hates x y
29	axiom ∀ x y, killed x y → ¬richer x y
30	axiom ∀ x, hates agatha x → ¬hates charles x
31	axiom ∀ x, x ≠ butler → hates agatha x
32	axiom ∀ x, ¬richer x agatha → hates butler x
33	axiom ∀ x, hates agatha x → hates butler x
34	axiom ¬(∃ x, ∀ y, lives y → hates x y)
35	axiom agatha ≠ butler
36	axiom Agatha = agatha
37	axiom ∀ x, (∃ e1 e2 y, person x ∧ _live_v_1 e1 x ∧ _in_p_dir e2 e1 y ∧ Dreadbury = y) ↔ lives x
38	axiom ∀ x y, (∃ e1, _kill_v_1 e1 x y) ↔ killed x y
39	axiom ∀ x y, (∃ e, _be_v_id e x y) ↔ x = y
40	axiom person Agatha
41	axiom person Charles
42	axiom ∀ x, _butler_n_1 x ↔ person x
43	axiom ∀ x, _butler_n_1 x ↔ x = butler
44	axiom ∀ x y z, ∃ x1 x2 e, implicit_conj x1 x x2 ∧ _and_c x2 y z ∧ _live_v_1 e x1 ↔ lives x ∧ lives y ∧ lives z
45	axiom ∀ x y z, ∃ x1 x2 e, implicit_conj x1 x x2 ∧ _and_c x2 y z ∧ _people_n_of x1 ↔ person x ∧ pers

In [14]:
print(RuleZ3(killed(agatha, agatha)))

⊢ killed agatha agatha


# MRS Logic

Consider the semantics of:

- Agatha is the only person who lives in Dreadbury.

or, we can't distinguish the two cases:

- Children who sing are happy.
- Children who are happy sing.

In [15]:
! killall ukb_wsd
%set_env ACE_GRAMMAR erg.dat
import mrs_logic
    
ctx = mrs_logic.get_current_context()
ctx.options['to_ulkb']['universe_type'] = a   # set the type of the (FOL) universe to "a"
ctx.options['to_ulkb']['drop_uvars'] = True   # delete "u" vars
ctx.options['to_ulkb']['drop_ivars'] = True   # delete "i" vars
ctx.options.to_ulkb.mk_q_exists = ['_the_q']

No matching processes belonging to you were found
env: ACE_GRAMMAR=erg.dat


In [24]:
txt = ['Someone who lives in Dreadbury killed Agatha.',
       'Agatha, the butler, and Charles live in Dreadbury, and are the only people who live therein.',
       'A killer always hates his victim, and is never richer than his victim',
       'Charles hates no one that Aunt Agatha hates.',
       'Agatha hates everyone except the butler',
       'The butler hates everyone not richer than Aunt Agatha',
       'The butler hates everyone Aunt Agatha hates',
       'No one hates everyone',
       'Agatha is not the butler']

In [25]:
sol = next(mrs_logic.solve(txt[0]))
form = sol.to_ulkb()
# print(sol.mrs)
# print(form)
# form.constants

x, y, e1, e2 = Variables('x', 'y', 'e1', 'e2', a)

Agatha    = Constant('Agatha', a)
Dreadbury = Constant('Dreadbury', a)
person    = Constant('person', FunctionType(a, bool))
_kill_v_1 = Constant('_kill_v_1', FunctionType(a, a, a, bool))
_in_p_dir = Constant('_in_p_dir', FunctionType(a, a, a, bool))
_live_v_1 = Constant('_live_v_1', FunctionType(a, a, bool))

new_axiom(equal(Agatha, agatha))
new_axiom(Forall(x, Iff(Exists(e1, e2, y, 
                               And(person(x), _live_v_1(e1, x), _in_p_dir(e2, e1, y), Equal(Dreadbury,y))), 
                        lives(x))))

new_axiom(Forall(x, y, Iff(Exists(e1, _kill_v_1(e1, x, y)), killed(x,y))))

show_axioms()
RuleZ3(form)

26	axiom ∃ x, lives x ∧ killed x agatha
27	axiom ∀ x, lives x → x = agatha ∨ x = butler ∨ x = charles
28	axiom ∀ x y, killed x y → hates x y
29	axiom ∀ x y, killed x y → ¬richer x y
30	axiom ∀ x, hates agatha x → ¬hates charles x
31	axiom ∀ x, x ≠ butler → hates agatha x
32	axiom ∀ x, ¬richer x agatha → hates butler x
33	axiom ∀ x, hates agatha x → hates butler x
34	axiom ¬(∃ x, ∀ y, lives y → hates x y)
35	axiom agatha ≠ butler
36	axiom Agatha = agatha
37	axiom ∀ x, (∃ e1 e2 y, person x ∧ _live_v_1 e1 x ∧ _in_p_dir e2 e1 y ∧ Dreadbury = y) ↔ lives x
38	axiom ∀ x y, (∃ e1, _kill_v_1 e1 x y) ↔ killed x y
39	axiom ∀ x y, (∃ e, _be_v_id e x y) ↔ x = y
40	axiom person Agatha
41	axiom person Charles
42	axiom ∀ x, _butler_n_1 x ↔ person x
43	axiom ∀ x, _butler_n_1 x ↔ x = butler
44	axiom ∀ x y z, ∃ x1 x2 e, implicit_conj x1 x x2 ∧ _and_c x2 y z ∧ _live_v_1 e x1 ↔ lives x ∧ lives y ∧ lives z
45	axiom ∀ x y z, ∃ x1 x2 e, implicit_conj x1 x x2 ∧ _and_c x2 y z ∧ _people_n_of x1 ↔ person x ∧ pers

⊢ ∃ x10, Dreadbury = x10 ∧ (∃ x3, (∃ e8 e9, person x3 ∧ _live_v_1 e8 x3 ∧ _in_p_dir e9 e8 x10) ∧ (∃ x16, Agatha = x16 ∧ (∃ e2, _kill_v_1 e2 x3 x16)))

In [26]:
sol = next(mrs_logic.solve(txt[1]))
form = sol.to_ulkb()

print(sol.mrs)
print(form)
form.constants



[ TOP: h0
  INDEX: e2
  RELS: < [ udef_q LBL: h4 ARG0: x3 RSTR: h5 BODY: h6 ]
          [ proper_q LBL: h7 ARG0: x8 RSTR: h9 BODY: h10 ]
          [ named LBL: h11 ARG0: x8 CARG: "Agatha" ]
          [ udef_q LBL: h13 ARG0: x14 RSTR: h15 BODY: h16 ]
          [ implicit_conj LBL: h17 ARG0: x3 ARG1: x8 ARG2: x14 ]
          [ _the_q LBL: h18 ARG0: x19 RSTR: h20 BODY: h21 ]
          [ _butler_n_1 LBL: h22 ARG0: x19 ]
          [ _and_c LBL: h23 ARG0: x14 ARG1: x19 ARG2: x24 ]
          [ proper_q LBL: h25 ARG0: x24 RSTR: h26 BODY: h27 ]
          [ named LBL: h28 ARG0: x24 CARG: "Charles" ]
          [ _live_v_1 LBL: h1 ARG0: e30 ARG1: x3 ]
          [ _in_p_dir LBL: h1 ARG0: e31 ARG1: e30 ARG2: x32 ]
          [ proper_q LBL: h33 ARG0: x32 RSTR: h34 BODY: h35 ]
          [ named LBL: h36 ARG0: x32 CARG: "Dreadbury" ]
          [ _and_c LBL: h1 ARG0: e2 ARG1: e30 ARG2: e38 ]
          [ _be_v_id LBL: h1 ARG0: e38 ARG1: x3 ARG2: x39 ]
          [ _the_q LBL: h40 ARG0: x39 RSTR: h41 BODY:

frozenset({Agatha : a,
           Charles : a,
           Dreadbury : a,
           _and_c : a → a → a → 𝔹,
           _be_v_id : a → a → a → 𝔹,
           _butler_n_1 : a → 𝔹,
           _in_p_dir : a → a → a → 𝔹,
           _live_v_1 : a → a → 𝔹,
           _only_a_1 : a → a → 𝔹,
           _people_n_of : a → 𝔹,
           _therein_p_dir : a → a → 𝔹,
           and : 𝔹 → 𝔹 → 𝔹,
           equal : a → a → 𝔹,
           exists : (a → 𝔹) → 𝔹,
           implicit_conj : a → a → a → 𝔹})

In [30]:
x, x1, x2, x3, w, z, e, e1, e2 = Variables('x', 'x1', 'x2', 'x3', 'w', 'z', 'e', 'e1', 'e2', a)

Charles      = Constant('Charles', a)
_butler_n_1  = Constant('_butler_n_1', FunctionType(a, bool))
_people_n_of = Constant('_people_n_of', FunctionType(a, bool))

implicit_conj = Constant('implicit_conj', FunctionType(a, a, a, bool))
_and_c        = Constant('_and_c', FunctionType(a, a, a, bool))
_be_v_id      = Constant('_be_v_id', FunctionType(a, a, a, bool))
_only_a_1     = Constant('_only_a_1', FunctionType(a, a, bool))

def ax1(p1,p2):
    return Forall(x, y, z, Exists(x1, x2, e, 
                                Iff(And(implicit_conj(x1, x, x2), _and_c(x2, y, z), p1(e, x1)),
                                    Exists(w, Implies(p2(w), Or(Equal(w,x), Equal(w,y), Equal(w,z)))))))

def ax1(p1,p2):
    return Forall(x, y, z, Exists(x1, x2, e, 
                                Iff(And(implicit_conj(x1, x, x2), _and_c(x2, y, z), p1(e, x1)),
                                    And(p2(x), p2(y), p2(z)))))

def ax2(p1,p2):
    return Forall(x, y, z, Exists(x1, x2, e, 
                                Iff(And(implicit_conj(x1, x, x2), _and_c(x2, y, z), p1(x1)),
                                    And(p2(x), p2(y), p2(z)))))

# semantics of only

new_axiom(Forall(x,y, Iff(Exists(e, _be_v_id(e, x, y)), Equal(x,y))))
new_axiom(person(Agatha))
new_axiom(person(Charles))
new_axiom(Forall(x, Iff(_butler_n_1(x), person(x))))
new_axiom(Forall(x, Iff(_butler_n_1(x), Equal(x,butler)))) # butler is not a entity but a set

new_axiom(ax1(_live_v_1,lives))
new_axiom(ax2(_people_n_of,person))

show_axioms()
RuleZ3(form)

26	axiom ∃ x, lives x ∧ killed x agatha
27	axiom ∀ x, lives x → x = agatha ∨ x = butler ∨ x = charles
28	axiom ∀ x y, killed x y → hates x y
29	axiom ∀ x y, killed x y → ¬richer x y
30	axiom ∀ x, hates agatha x → ¬hates charles x
31	axiom ∀ x, x ≠ butler → hates agatha x
32	axiom ∀ x, ¬richer x agatha → hates butler x
33	axiom ∀ x, hates agatha x → hates butler x
34	axiom ¬(∃ x, ∀ y, lives y → hates x y)
35	axiom agatha ≠ butler
36	axiom Agatha = agatha
37	axiom ∀ x, (∃ e1 e2 y, person x ∧ _live_v_1 e1 x ∧ _in_p_dir e2 e1 y ∧ Dreadbury = y) ↔ lives x
38	axiom ∀ x y, (∃ e1, _kill_v_1 e1 x y) ↔ killed x y
39	axiom ∀ x y, (∃ e, _be_v_id e x y) ↔ x = y
40	axiom person Agatha
41	axiom person Charles
42	axiom ∀ x, _butler_n_1 x ↔ person x
43	axiom ∀ x, _butler_n_1 x ↔ x = butler
44	axiom ∀ x y z, ∃ x1 x2 e, implicit_conj x1 x x2 ∧ _and_c x2 y z ∧ _live_v_1 e x1 ↔ lives x ∧ lives y ∧ lives z
45	axiom ∀ x y z, ∃ x1 x2 e, implicit_conj x1 x x2 ∧ _and_c x2 y z ∧ _people_n_of x1 ↔ person x ∧ pers

⊢ ∃ x32, Dreadbury = x32 ∧ (∃ x14, (∃ x24, Charles = x24 ∧ (∃ x19, _butler_n_1 x19 ∧ _and_c x14 x19 x24)) ∧ (∃ x39, (∃ e44 e46 e47, _only_a_1 e44 x39 ∧ _people_n_of x39 ∧ _live_v_1 e46 x39 ∧ _therein_p_dir e47 e46) ∧ (∃ x3, (∃ x8, Agatha = x8 ∧ implicit_conj x3 x8 x14) ∧ (∃ e30 e31 e2 e38, _live_v_1 e30 x3 ∧ _in_p_dir e31 e30 x32 ∧ _and_c e2 e30 e38 ∧ _be_v_id e38 x3 x39))))